In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
data = pd.read_csv('small_sample2.csv')
data=data.dropna()

In [4]:
dates = data['time'].values
data['time'] = pd.to_datetime(data['time'])
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month
data['day'] = data['time'].dt.day
data = data.drop(columns=['time'])

In [5]:

features = data[['depth', 'year', 'month', 'day', 'latitude', 'longitude']].values
targets = data[['sea_water_temperature', 'sea_water_practical_salinity']].values


In [6]:
data.to_csv('data.csv', index=False)

In [7]:
scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(features)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(targets)

X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
dates_train, dates_temp = train_test_split(dates, test_size=0.2, random_state=42)
dates_val, dates_test = train_test_split(dates_temp, test_size=0.5, random_state=42)

In [8]:
"""
df_X_train = pd.DataFrame(X_train)
df_X_val = pd.DataFrame(X_val)
df_X_test = pd.DataFrame(X_test)
df_y_train = pd.DataFrame(y_train)
df_y_val = pd.DataFrame(y_val)
df_y_test = pd.DataFrame(y_test)

df_X_train.to_csv('X_train.csv', index=False)
df_X_val.to_csv('X_val.csv', index=False)
df_X_test.to_csv('X_test.csv', index=False)
df_y_train.to_csv('y_train.csv', index=False)
df_y_val.to_csv('y_val.csv', index=False)
df_y_test.to_csv('y_test.csv', index=False)
"""


"\ndf_X_train = pd.DataFrame(X_train)\ndf_X_val = pd.DataFrame(X_val)\ndf_X_test = pd.DataFrame(X_test)\ndf_y_train = pd.DataFrame(y_train)\ndf_y_val = pd.DataFrame(y_val)\ndf_y_test = pd.DataFrame(y_test)\n\ndf_X_train.to_csv('X_train.csv', index=False)\ndf_X_val.to_csv('X_val.csv', index=False)\ndf_X_test.to_csv('X_test.csv', index=False)\ndf_y_train.to_csv('y_train.csv', index=False)\ndf_y_val.to_csv('y_val.csv', index=False)\ndf_y_test.to_csv('y_test.csv', index=False)\n"

In [9]:
class CombinedDataset(Dataset):
    def __init__(self, features, targets, seq_length=None):
        self.features = features
        self.targets = targets
        self.seq_length = seq_length

        if seq_length is not None:
            self.sequences = self._create_sequences(features, seq_length)
            self.sequence_targets = targets[seq_length:]
            
        else:
            self.sequences = features
            self.sequence_targets = targets

    def _create_sequences(self, data, seq_length):
        sequences = []
        for i in range(len(data) - seq_length):
            sequences.append(data[i:i+seq_length])
        return sequences

    def __len__(self):
        return len(self.sequence_targets)

    def __getitem__(self, index):
        if self.seq_length is not None:
            return (self.sequences[index], self.sequence_targets[index])
        else:
            return (self.features[index], self.targets[index])


In [10]:

seq_length = 512
train_dataset_seq = CombinedDataset(X_train, y_train, seq_length)
val_dataset_seq = CombinedDataset(X_val, y_val, seq_length)


train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))


In [11]:
from torch.utils.data import DataLoader

batch_size = 512
num_workers = 4 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [12]:
class TransformerModel(nn.Module):
    def __init__(self):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=6, nhead=2, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=6)
        self.decoder = nn.Linear(6, 2) 

    def forward(self, src):
        output = self.transformer_encoder(src)
        if output.dim() == 3:
            output = self.decoder(output[:, -1, :])
        elif output.dim() == 2:
            output = self.decoder(output)
        return output

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.1, verbose=True)

In [14]:
def validate(model, loader, criterion, device, verbose=False):
    model.eval()
    total_loss = 0.0
    predictions, actuals = [], []

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            if verbose:
                print(f"Batch {batch_idx} input shape: {inputs.size()}")

            if len(inputs.size()) == 2:
                inputs = inputs.unsqueeze(1)

            inputs, targets = inputs.to(device), targets.to(device)

            try:
                outputs = model(inputs.float())
                loss = criterion(outputs, targets.float())
                total_loss += loss.item() * inputs.size(0)

                predictions.extend(outputs.detach().cpu().tolist())
                actuals.extend(targets.detach().cpu().tolist())

            except RuntimeError as e:
                print(f"Error in batch {batch_idx}: {e}")
                break

    average_loss = total_loss / len(loader.dataset)
    if verbose:
        print(f'Average validation loss: {average_loss:.4f}')

    return average_loss, predictions, actuals

num_epochs = 100
best_val_loss = np.inf
patience, trials = 20, 0

In [15]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, targets.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)


    val_loss, _, _ = validate(model, val_loader, criterion, device)
    scheduler.step(val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_transformer_model.pth')
    else:
        trials += 1
        if trials >= patience:
            print("Early stopping triggered.")
            break

Epoch [1/100], Loss: 1.0230, Val Loss: 0.9945
Epoch [2/100], Loss: 1.0070, Val Loss: 0.9918
Epoch [3/100], Loss: 1.0072, Val Loss: 0.9930
Epoch [4/100], Loss: 1.0073, Val Loss: 0.9920
Epoch [5/100], Loss: 1.0068, Val Loss: 0.9922
Epoch [6/100], Loss: 1.0075, Val Loss: 0.9925
Epoch [7/100], Loss: 1.0074, Val Loss: 0.9925
Epoch [8/100], Loss: 1.0066, Val Loss: 0.9944
Epoch [9/100], Loss: 1.0066, Val Loss: 0.9819
Epoch [10/100], Loss: 1.0087, Val Loss: 0.9931
Epoch [11/100], Loss: 1.0076, Val Loss: 0.9922
Epoch [12/100], Loss: 1.0066, Val Loss: 0.9917
Epoch [13/100], Loss: 1.0068, Val Loss: 0.9928
Epoch [14/100], Loss: 1.0060, Val Loss: 0.9931
Epoch [15/100], Loss: 1.0068, Val Loss: 0.9918
Epoch [16/100], Loss: 1.0060, Val Loss: 0.9930
Epoch [17/100], Loss: 1.0064, Val Loss: 0.9917
Epoch [18/100], Loss: 1.0065, Val Loss: 0.9921
Epoch [19/100], Loss: 1.0064, Val Loss: 0.9924
Epoch 00020: reducing learning rate of group 0 to 1.0000e-03.
Epoch [20/100], Loss: 1.0062, Val Loss: 0.9921
Epoch [

In [16]:
average_val_loss, val_predictions, val_actuals = validate(model, val_loader, criterion, device, verbose=True)
print(f'Average validation loss: {average_val_loss:.4f}')


Batch 0 input shape: torch.Size([512, 6])
Batch 1 input shape: torch.Size([512, 6])
Batch 2 input shape: torch.Size([512, 6])
Batch 3 input shape: torch.Size([512, 6])
Batch 4 input shape: torch.Size([512, 6])
Batch 5 input shape: torch.Size([512, 6])
Batch 6 input shape: torch.Size([130, 6])
Average validation loss: 0.9927
Average validation loss: 0.9927


In [17]:
def inverse_transform(scaler, data, column_indices):

    data = np.array(data)
    dummy = np.zeros((data.shape[0], len(scaler.scale_)))
    dummy[:, column_indices] = data
    return scaler.inverse_transform(dummy)[:, column_indices]


predictions_rescaled = inverse_transform(scaler_y, val_predictions, [0, 1])
actuals_rescaled = inverse_transform(scaler_y, val_actuals, [0, 1])


In [18]:
rmse_temp = np.sqrt(mean_squared_error(actuals_rescaled[:, 0], predictions_rescaled[:, 0]))
mae_temp = mean_absolute_error(actuals_rescaled[:, 0], predictions_rescaled[:, 0])

rmse_salinity = np.sqrt(mean_squared_error(actuals_rescaled[:, 1], predictions_rescaled[:, 1]))
mae_salinity = mean_absolute_error(actuals_rescaled[:, 1], predictions_rescaled[:, 1])

print(f'Average Validation Loss: {average_val_loss:.4f}')
print(f'Temperature - RMSE: {rmse_temp:.4f}, MAE: {mae_temp:.4f}')
print(f'Salinity - RMSE: {rmse_salinity:.4f}, MAE: {mae_salinity:.4f}')


Average Validation Loss: 0.9927
Temperature - RMSE: 0.7366, MAE: 0.6043
Salinity - RMSE: 0.6943, MAE: 0.4647


In [19]:
df = pd.DataFrame({
    'Actual Temperature': actuals_rescaled[:, 0],
    'Predicted Temperature': predictions_rescaled[:, 0],
    'Actual Salinity': actuals_rescaled[:, 1],
    'Predicted Salinity': predictions_rescaled[:, 1]
})


csv_file_path = 'actuals_vs_predictions.csv'
df.to_csv(csv_file_path, index=False)